# Processing

## Preparation

In [2]:
import rasterio
import queue
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
import threading
from IPython.display import clear_output
from collections import Counter, namedtuple
from src.utils import get_data_dir, benchmark_context


# Convenient access to data directory, is a namedtuple with folder names as attributes
DIRS = get_data_dir(str(Path('data').resolve()))

# Many functions of the processing pipeline are multi-threaded this attribute controls
# max number of threads
THREADLIMIT = 12

WGS84 = {'init': 'epsg:4326'}


def read_raster(item):
    """
    Helper method to return a raster file as a opened instance of
    rasterio.io.DatasetReader in read mode.
    
    :param item: str, pathlib.Path or rasterio.io.DatasetReader
        Should be the path to the raster file on filesystem as a string
        or pathlib.Path object. If item is a instance of DatasetReader
        the function returns immediately.
    :return: rasterio.io.DatasetReader
        Retruns an instance of rasterio.io.DatasetReader in read mode.
    """
    if isinstance(item, rasterio.io.DatasetReader):
        return item
    else:
        try:
            path = str(item)  # Cast pathlib.Path to string
            return rasterio.open(path, 'r')
        except:
            msg = 'Attr {}, Type {} is not a valid raster file'.format(item, type(item))
            raise ValueError(msg)


def binary_jaccard(arr1, arr2, return_matrix=False):
    """
    Calculates the Jaccard Index (JI) of two equal sized binary arrays or vectors.
    If return_matrix is set to true the method provides the JI and the necessary 
    calculation matrix as a named tuple. Attention, this method does not work in-place!
    
    :param arr1, arr2: numpy.ndarray, list, tuple
        Both array alike objects sized in equal dimensions should contain exclusively 
        binary data (1,0). 
    :param return_matrix: boolean
        Optional, a boolean value determining the return of the calculation matrix. 
    :return: float OR (float, namedtuple(m11, m01, m10, m00))
        Defaultly, the method returns only the JI if, return_matrix is set to true the 
        method returns the JI and the computation matrix.
        The Matrix contains the following attributes:
        m11 = total number of attributes where arr1 == 1 and arr2 == 1
        m10 = total number of attributes where arr1 == 1 and arr2 == 0
        m01 = total number of attributes where arr1 == 0 and arr2 == 1
        m00 = not required, set to 0
    """
    A, B = np.array(arr1, dtype=np.int8), np.array(arr2, dtype=np.int8)
    
    if np.sum(np.logical_or(A<0,A>1)) != 0 or np.sum(np.logical_or(B<0,B>1)) != 0:
        raise ValueError('Attributes should contain only binary values')
  
    C = A + B
    a = (B - C) + B  # a = (A - C) + A, m10 = a == 1
    b = (A - C) + A  # b = (B - C) + B, m01 = b == 1

    # Total number of attributes where A == 1 and B == 1
    m11 = np.sum(C==2)
    # Total number of attributes where A == 1 and B == 0
    m10 = np.sum(a==-1)
    # Total number of attributes where A == 0 and B == 1
    m01 = np.sum(b==-1)
    
    jaccard = m11 / (m10 + m01 + m11)
    
    if return_matrix:
        Matrix = namedtuple('Matrix', 'm11 m10 m01 m00')
        return jaccard, Matrix(m11, m10, m01, 0)
    return jaccard


def simple_matching_coefficient(arr1, arr2, return_matrix=False):
    """
    Calculates the Simple Matching Coefficient (SMC) of two equal sized arrays or vectors.
    If return_matrix is set to true the method provides the SMC and the necessary calculation 
    matrix as a named tuple. Attention, this method does not work in-place!
    
    :param arr1, arr2: numpy.ndarray, list, tuple
        Both array alike objects sized in equal dimensions should contain exclusively 
        binary data (1,0).
    :param return_matrix: boolean
        Optional, a boolean value determining the return of the calculation matrix.
    :return: float OR (float, namedtuple(m11, m01, m10, m00))
        Defaultly, the method returns only the SMC, if return_matrix is
        set to true the method returns the SMC and the computation matrix.
        The Matrix contains the following attributes:
        m11 = total number of attributes where arr1 == 1 and arr2 == 1
        m10 = total number of attributes where arr1 == 1 and arr2 == 0
        m01 = total number of attributes where arr1 == 0 and arr2 == 1
        m00 = total number of attributes where arr1 == 0 and arr2 == 0
    """
    _, matrix = binary_jaccard(arr1, arr2, True)
    A = np.array(arr1, dtype=np.int8)
    
    # Total number of attributes where A == 0 and B == 0
    m00 = A.size - sum(matrix)
    
    smc = (matrix.m11 + m00) / A.size

    if return_matrix:
        matrix = matrix._replace(m00=m00)
        return smc, matrix
    return smc

## Class harmonization

In [3]:
def worker(gl30, cover, queue, *args):
    r1 = read_raster(gl30)
    r2 = read_raster(cover)
    
    d1 = r1.read(1)
    d2 = r2.read(1)
    
    r1.close()
    r2.close()
    
    d1[d1==20] = 1
    d1[d1==50] = 1
    d1[d1!=1] = 0
    
    tmp = d2.copy()
    
    args = list(args)
    for i in [0, 10, 20, 30]:
        tmp[d2<=i] = 0
        tmp[d2>i] = 1

        args.append(binary_jaccard(d1, tmp))
        args.append(simple_matching_coefficient(d1, tmp))
   
    queue.put(args)


mask = gpd.read_file(str(DIRS.masks / 'final_region_mask.shp'))
q = queue.Queue()
threads = []
for idx, val in mask.iterrows():
    if len(threads) == 4:
        [thread.join() for thread in threads]
        threads = []
        clear_output()
        print('{}% of 100%'.format(((idx+1)/len(mask))*100))
    
    thread = threading.Thread(target=worker, args=(DIRS.proc/val.gl30_00, DIRS.proc/val.cover, q, val.key, val.index_righ))
    thread.start()
    threads.append(thread)

99.6551724137931% of 100%


In [4]:
vals = []
while not q.empty():
    vals.append(q.get())

df = pd.DataFrame(vals)
df.columns = ['tile', 'region', 'jc0', 'smc0', 'jc10', 'smc10', 'jc20', 'smc20', 'jc30', 'smc30']
df.to_csv(DIRS.ana/'class_harmon_wet.csv', float_format='%.4f')